In [1]:
from tensorflow.keras.models import load_model
loaded_model = load_model('model.h5')

2022-02-15 21:24:08.504963: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-15 21:24:08.505264: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


In [2]:
import cv2
import numpy as np
def test_model(img): #img = file_name(str)
    img = cv2.imread(img)
    img = cv2.resize(img, (32, 32))
    img = img/255.0
    l = []
    l.append(img)
    l = np.asarray(l)
    prediction = np.argmax(loaded_model.predict(l))
    result = ''
    if prediction == 0:
        result = 'prediction: apple'
    elif prediction == 1:
        result = 'prediction: banana'
    elif prediction == 2:
        result = 'prediction: capsicum'
    elif prediction == 3:
        result = 'prediction: corn'
    else:
        result = 'prediction: orange'
    return result

In [3]:
import serial
import time
cam = cv2.VideoCapture(0)

cv2.namedWindow("test")
img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    text = ""
    if k%256 == 27:
        # ESC pressed
        #apple is 25 baht, banana is 20 baht, capsicum is 15 baht, corn is 10 baht, ornage is 5 baht.
        text = "Done"
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        img_counter += 1
        result = test_model(img_name)
        if result == 'prediction: apple':
            text = "Apple 25"
        elif result == 'prediction: banana':
            text = "Banana 20"
        elif result == 'prediction: capsicum':
            text = "Capsicum 15"
        elif result == 'prediction: corn':
            text = "Corn 10"
        else:
            text = "Orange 5"
    serialcomm = serial.Serial('/dev/cu.usbmodem14201', 9600)  # tf are ports really?
    serialcomm.timeout = 1  # huh?
    if text == 'Done':
        print('finished program')
        break
    serialcomm.write(text.encode())
    time.sleep(0.01)

serialcomm.close()
          
cam.release()

cv2.destroyAllWindows()

KeyboardInterrupt: 

In [3]:
import serial
import time


serialcomm = serial.Serial('/dev/cu.usbmodem14201', 9600)  # tf are ports really?
serialcomm.timeout = 1  # huh?

while True:
    i = input("input: ").strip()
    if i == 'done':
        print('finished program')
        break
    serialcomm.write(i.encode())
    time.sleep(0.5)
    print(serialcomm.readline().decode('ascii'))

serialcomm.close()

input: Apple 25

input: done
finished program
